In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shap
import mlflow

from scipy.stats import entropy, shapiro, chi2_contingency, f_oneway
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import STL
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from prophet import Prophet

from sklearn.cluster import KMeans
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, ConfusionMatrixDisplay, roc_curve, auc, precision_recall_curve, average_precision_score, mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import label_binarize

from sklearn.manifold import TSNE
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [ ]:
def load_data(path: str) -> pd.DataFrame:
    df = pd.read_csv(path)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    return df

In [ ]:
def plot_daily_weekly_cycles(df: pd.DataFrame, customer_id: int):
    cust = df[df['MeterID'] == customer_id].set_index('timestamp')
    # daily cycle
    daily = cust['LastAverageValueOfImportActivePower'].groupby(cust.index.time).mean()
    plt.figure()
    plt.plot(daily.index.astype(str), daily.values)
    plt.xticks(rotation=90)
    plt.title(f"Daily cycle for customer {customer_id}")
    plt.xlabel('Time of day')
    plt.ylabel('Average Power')
    plt.show()
    # weekly cycle
    weekly = cust['LastAverageValueOfImportActivePower'].groupby(cust.index.dayofweek).mean()
    plt.figure()
    plt.bar(weekly.index, weekly.values)
    plt.title(f"Weekly cycle for customer {customer_id}")
    plt.xlabel('Day of week')
    plt.ylabel('Average Power')
    plt.show()


def compute_entropy(ts: pd.Series, bins: int = 50) -> float:
    counts, _ = np.histogram(ts, bins=bins, density=False)
    probs = counts / counts.sum()
    return entropy(probs, base=2)


def plot_entropy_all(df: pd.DataFrame):
    ent = df.groupby('MeterID')['LastAverageValueOfImportActivePower'].apply(compute_entropy)
    ent.plot(kind='hist', bins=20)
    plt.title('Distribution of Entropy across customers')
    plt.xlabel('Entropy')
    plt.show()


def cluster_load_shapes(df: pd.DataFrame, n_clusters: int = 2):
    # extract daily load shapes
    pivot = df.pivot_table(values='LastAverageValueOfImportActivePower', index='MeterID', columns=df['timestamp'].dt.hour * 4 + df['timestamp'].dt.minute // 15)
    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(pivot.fillna(0))
    return kmeans.labels_

In [ ]:
df = load_data('../data/smart_meter_data_with_ntl.csv')
plot_daily_weekly_cycles(df, customer_id=1)
plot_entropy_all(df)

In [ ]:
labels = cluster_load_shapes(df)
df['cluster'] = df['MeterID'].map(dict(zip(df['MeterID'].unique(), labels)))

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
_, axis = plt.subplots(nrows=2, ncols=2, figsize=(15.5, 7.5))

# Voltage and current
sns.histplot(
    data=df,
x='LastAverageValueOfCurrent', ax=axis[0, 0], hue='NTL_Type',
)

sns.histplot(
    data=df,
x='LastAverageValueOfVoltageL', ax=axis[0, 1], hue='NTL_Type'
)

# power
sns.histplot(
    data=df,
x='LastAverageValueOfPFTotal', ax=axis[1, 0], hue='NTL_Type'
)

sns.histplot(
    data=df,
x='LastAverageValueOfImportActivePower', ax=axis[1, 1], hue='NTL_Type'
)

plt.tight_layout()

## Left Plot: `LastAverageValueOfCurrent`

- **Shape**: Strong right skew (long tail to the right).

### Modes:
- A very strong peak near **0 A** (likely many devices are off or in standby).
- A secondary mode around **3-4 A**, which likely represents typical operating current.

### Interpretation:
- Most devices either draw very low current (possibly idle) or a moderate current.
- The long tail up to ~**17 A** may represent higher-load appliances or noise/outliers.

### Possible NTL (Non-Technical Loss) Insight:
- If current is high but corresponding energy usage is missing or low, this could suggest **bypass or tampering**.

---

## Right Plot: `LastAverageValueOfVoltageL`

- **Shape**: Approximately Gaussian (normal distribution).
- **Centered around**: ~**215V** (with a range from ~**200V to 230V**).

### Interpretation:
- This is expected for a well-functioning low-voltage distribution system  
  (in many countries, **220V ±10%** is acceptable).
- Less suspicious behavior here unless voltage drops/cut-offs correlate with **abnormal current patterns**.

In [ ]:
# take log of the above plot

_, axis = plt.subplots(nrows=2, ncols=2, figsize=(15.5, 7.5))

# Voltage and current
sns.histplot(
    data=df,
x=np.log10(df['LastAverageValueOfCurrent']), ax=axis[0, 0], hue='NTL_Type',
)

sns.histplot(
    data=df,
x=np.log10(df['LastAverageValueOfVoltageL']), ax=axis[0, 1], hue='NTL_Type'
)

# power
sns.histplot(
    data=df,
x=np.log10(df['LastAverageValueOfPFTotal']), ax=axis[1, 0], hue='NTL_Type'
)

sns.histplot(
    data=df,
x=np.log10(df['LastAverageValueOfImportActivePower']), ax=axis[1, 1], hue='NTL_Type'
)

plt.tight_layout()

In [ ]:
plt.figure(figsize=(15, 7.5))

ax = sns.scatterplot(
        data=df,
        x='LastAverageValueOfCurrent', y='LastAverageValueOfImportActivePower',
        hue='NTL_Type', alpha=0.5
    )

plt.xscale('log')
plt.yscale('log')

sns.move_legend(ax, "upper right")

In [ ]:
plt.figure(figsize=(15, 7.5))
ax = sns.scatterplot(
    data=df,
    x='LastAverageValueOfCurrent',
    y='LastAverageValueOfImportActivePower',
    hue='NTL_Type',
    alpha=0.5,
    palette='colorblind'
)

sns.move_legend(ax, 'upper left')

In [ ]:
plt.figure(figsize=(15, 7.5))
g = sns.FacetGrid(df, col='NTL_Type')
g.map_dataframe(sns.scatterplot, x='LastAverageValueOfCurrent', y='LastAverageValueOfImportActivePower',
                alpha=0.5)

In [ ]:
numerics = df.select_dtypes(include='number').drop(columns=['MeterID', 'Phase', 'cluster', 'ReactiveCumulativeEnergyImport(+R)'])
sns.heatmap(numerics.corr(), annot=True, cmap='coolwarm')